In [1]:
import pandas as pd

In [2]:
projects_time_series = pd.read_pickle('../../data/02_prepared_data/projects_time_series.pkl')
projects_time_series

,project_id,name,language,created_at,year,month,months_from_create
13,1,akka,Scala,2009-02-16 12:51:54,2009,2,0
14,1,akka,Scala,2009-02-16 12:51:54,2009,3,1
15,1,akka,Scala,2009-02-16 12:51:54,2009,4,2
16,1,akka,Scala,2009-02-16 12:51:54,2009,5,3
17,1,akka,Scala,2009-02-16 12:51:54,2009,6,4
...,...,...,...,...,...,...,...
7820279,108739,rails,Ruby,2013-09-27 17:53:25,2013,12,3
7820348,108740,rails,Ruby,2013-09-26 14:38:42,2013,9,0
7820349,108740,rails,Ruby,2013-09-26 14:38:42,2013,10,1
7820350,108740,rails,Ruby,2013-09-26 14:38:42,2013,11,2


Usunięcie zbędnych kolumn i zresetowanie indeksów

In [3]:
projects_time_series = projects_time_series.reset_index().drop(columns={'index', 'name', 'created_at'})

## Przykład - połączenie danych commitów i wypełnienie braków danych

In [4]:
new_commits = pd.read_pickle('../../data/02_prepared_data/new_commits.pkl')
new_commits.head()

,project_id,year,month,new_commits,total_commits
0,2,2010,5,43,43
1,2,2010,6,4,47
2,2,2010,7,3,50
3,2,2010,8,1,51
4,2,2010,9,1,52


In [5]:
time_series_with_commits_info = pd.merge(projects_time_series, new_commits, on=['project_id', 'year', 'month'], how='left', sort=False)
time_series_with_commits_info.isnull().sum()

project_id                  0
language                    0
year                        0
month                       0
months_from_create          0
new_commits           1793145
total_commits         1793145
dtype: int64

Jaki procent wierszy ma jakiekolwiek commity

In [6]:
(1810601 - 1793145)/1810601 * 100

0.964099765768383

Obsługa brakujących danych:
- dla kolumny `new_commits` w miejscu braków uzupełniamy zerem
- dla kolumny `total_commits` w miejscu braków uzupełniamy wartością z wiersza wyżej

In [7]:
time_series_with_commits_info['new_commits'].fillna(value=0, inplace=True)
time_series_with_commits_info.isnull().sum()

project_id                  0
language                    0
year                        0
month                       0
months_from_create          0
new_commits                 0
total_commits         1793145
dtype: int64

In [8]:
time_series_with_commits_info['new_commits'] = time_series_with_commits_info['new_commits'].astype('int64')
time_series_with_commits_info

,project_id,language,year,month,months_from_create,new_commits,total_commits
0,1,Scala,2009,2,0,0,NaN
1,1,Scala,2009,3,1,0,NaN
2,1,Scala,2009,4,2,0,NaN
3,1,Scala,2009,5,3,0,NaN
4,1,Scala,2009,6,4,0,NaN
...,...,...,...,...,...,...,...
1810596,108739,Ruby,2013,12,3,0,NaN
1810597,108740,Ruby,2013,9,0,0,NaN
1810598,108740,Ruby,2013,10,1,0,NaN
1810599,108740,Ruby,2013,11,2,0,NaN


In [9]:
pd.isna(time_series_with_commits_info.loc[0]['total_commits'])

True

In [10]:
time_series_with_commits_info['total_commits'] = \
    time_series_with_commits_info.groupby('project_id')['total_commits'].ffill().fillna(0)
time_series_with_commits_info['total_commits'] = time_series_with_commits_info['total_commits'].astype('int64')
time_series_with_commits_info.isnull().sum()

project_id            0
language              0
year                  0
month                 0
months_from_create    0
new_commits           0
total_commits         0
dtype: int64

Sprawdzenie poprawności wypełnienia braków

In [11]:
time_series_with_commits_info[
    (time_series_with_commits_info['project_id'] == 5)]

,project_id,language,year,month,months_from_create,new_commits,total_commits
165,5,C++,2010,1,0,0,0
166,5,C++,2010,2,1,141,141
167,5,C++,2010,3,2,10,151
168,5,C++,2010,4,3,0,151
169,5,C++,2010,5,4,87,238
170,5,C++,2010,6,5,227,465
171,5,C++,2010,7,6,250,715
172,5,C++,2010,8,7,230,945
173,5,C++,2010,9,8,179,1124
174,5,C++,2010,10,9,176,1300
